# Gene Set and Pathway
![](./images/Module3/Workflow3.jpg)

Differential expression (DE) analysis typically yields a list of genes or proteins. Our intention is to use such lists to gain novel insights about genes and proteins that may have roles in a given
phenomenon, phenotype or disease progression. However, in many cases, gene lists generated from DE analysis are
difficult to interpret due to their large size and lack of useful annotations. Hence, pathway analysis (also known
as gene set analysis or over-representation analysis), aims to reduce the complexity of interpreting gene lists
via mapping the listed genes to known (i.e. annotated) biological pathways, processes and functions.
This learning submodule introduces common curated biological databases including Gene Ontology (GO), the
Kyoto Encyclopedia of Genes and Genomes (KEGG) and the database of reactions, pathways and biological processes (REACTOME).

## Learning Objectives:
1. Introduction to Ontology and Gene Ontology.
2. Introduction to KEGG Pathway Database.
3. Introduction to REACTOME Pathway Database
4. Download terms, pathway gene set from GO and KEGG.
5. Save results to GMT file format.

In [1]:
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule3-1.html" width=100% height=250></iframe>')

## Ontology and Gene Ontology
### Overview
In this section, we will learn about the concept of gene ontology in bioinformatics. Ontology is a set of concepts and categories defined by a shared vocabulary to denote  properties of the concepts, as well as the relationships between the concepts.
Ontology plays an important role in the field of bioinformatics. Ontology enables unambiguous communication e.g.,
a way to understand different groups’ annotations of various genomes. Also, it allows the knowledge to be structured to perform automated analyses by computer programs.

The Gene Ontology (GO) database defines a structured, common, and controlled vocabulary to describe attributes of genes and gene products
across organisms. Collaboration is key to build a consensus vocabulary. But the term gene ontology, or GO, is commonly used
to refer to both the terms as well as the associations between genes, which is sometimes a source of potential confusion. In order to avoid this, here we will use the term “GO”
to describe the set of terms and their hierarchical structure and “GO annotations” to describe the set of associations between
genes and GO terms. The GO is divided into three categories to describe the genes and gene products from three different
angles: Molecular Function, Biological Process, and Cellular Component.

The structure of GO can be described in terms of directed acyclic graphs (DAGs), where each GO term is a node,
and the relationships between the terms are edges. GO is loosely hierarchical, with ‘child’ terms
being more specialized than their ‘parent’ terms, but unlike a strict hierarchy, a term may have more than one parent
term (note that the parent/child model does not hold for all types of relations). The structure of the controlled
vocabularies is intended to reflect true, biological relationships. In contrast to strict hierarchies, DAGS allows
multiple relationships between a more granular (child) term and a more general parent term. The relationship between
terms affects how queries are made. For example, a query for all genes with binding activity would include transcription
factors as well as genes with other types of binding activity (such as protein binding, ligand binding). The illustration
of the category and structure of GO is shown in the figure below:

![](./images/Module3/GO_Structure.jpg)
*(Source: https://www.ebi.ac.uk/, http://geneontology.org/)*

### Gene ontology relationship
In DAGs graph, *terms* are represented as *nodes* and *relations* (also known as *object properties*) between the *terms*
are *edges*. There are commonly used relationships in GO such as *`is a`* (is a *`subtype of`*), *`part of`, `has part`, `regulates`,
`negatively regulates` and `positively regulates`*. All terms (except for the root terms representing each aspect) have a sub-class relationship to another term.

Examples:

> **Example:**
>  
> **GO:1904659:glucose transport** *`is a`* **GO:0015749:monosaccharide transport**.
>  
> The *`is a`* relation forms the basic structure of GO. If we say A *`is a`* B, we mean that node A is a subtype of node B


> **Example:**
>  
> **GO:0031966:mitochondrial membrane** *`is part of`* **GO:0005740:mitochondrial envelope**
>  
> The *`part of`* relation is used to represent part-whole relationships. A *`part of`* relation would only be added between
A and B if B is **necessarily** *`part of`* A: wherever B exists, it is as *`part of`* A, and the presence of the B implies
the presence of A. However, given the occurrence of A, we cannot say for certain that B exists.


> **Example:**
>  
> **GO:0098689:latency-replication decision** *`regulates`* **GO:0019046:release from viral latency**
>  
> A relation that describes a case in which one process directly affects the manifestation of another process or quality,
> i.e. the former *`regulates`* the latter.


A more specific case with more nodes and edges can be seen in the figure below:
<br>
![](./images/Module3/GO_Relation.jpg)

*(Source: https://advaitabio.com/)* <br>
For more technical information about relations and their properties used in GO and other ontologies, see the
<a href="https://obofoundry.org/ontology/ro.html">OBO Relations Ontology (RO)</a>


In [2]:
#Run the following command to take the quiz
IRdisplay::display_html('<iframe src="Quizzes/Quiz_Submodule3.html" width=100% height=250></iframe>')

### GO storage file formats
GO terms are updated monthly in the following formats:
* OBO 1.4 files are human-readable (in addition to machine-readable) and can be opened in any text editor.
* OWL files can be read by <a href="https://protege.stanford.edu/">Protégé</a> text editor.

 In this learning submodule, we will only use ".OBO" to obtain GO terms. The OBO file format is for representing ontologies and controlled vocabularies. The format itself attempts to achieve the following goals:
 * Human readability
 * Ease of parsing
 * Extensibility
 * Minimal redundancy

The file structure is shown in the following figure.


![](./images/Module3/OBO_Format.jpg)

The OBO file has a header, which is an unlabeled section at the beginning of the document. The header ends when the first term is encountered. Next, terms are represented in a labeled section with the tag *[Term]*. Under each term, we can find other information such as term ID, official name, category (namespace), term definition, synonym and relation to other GO terms.

At this step, we still don't know what genes are related to which GO terms. In order to retrieve custom sets of gene ontology annotations for any list of genes from organisms, NCBI has published a Gene2GO database that obtains GO terms and the gene symbols related to those GO terms. The database can be retrieved from <a href="https://ftp.ncbi.nih.gov/gene/DATA/gene2go.gz">here</a>. The Gene2GO database can be viewed using a text editor, the file structure is presented in the figure below:

![](./images/Module3/Gene2GO.jpg)

The OBO and Gene2GO databases will be used in combination to obtain GO terms and related genes for enrichment analysis.

### Retrieving GO terms from the DE gene list
This section focuses on downloading related GO terms based on the DE gene list obtained from the DE analysis in the previous submodules.
Here, we will use `topGO` and `hgu133plus2.db` R packages to obtain GO terms. The `topGO` package has built-in functions that use Gene2GO databases to retrieve GO terms from the gene ID given by DE analysis. Since the dataset we used in submodule 02 was generated for humans, we will use `hgu133plus2.db` database to map probeIDs to gene Symbols.
The installation process of two packages can be done by the script below:

In [3]:
# Installation of topGO and hgu133plus2.db package
suppressMessages({
    if (!require("BiocManager", quietly = TRUE)) {
        install.packages("BiocManager")
    }
    suppressWarnings(BiocManager::install("topGO", update = F))
    suppressWarnings(BiocManager::install("hgu133plus2.db", update = F))
    suppressWarnings(BiocManager::install("AnnotationDbi", update = F))
})

In [4]:
# Loading the library
suppressPackageStartupMessages({
    suppressWarnings(library("topGO"))
    suppressWarnings(library("hgu133plus2.db"))
    suppressWarnings(library("AnnotationDbi"))
})


groupGOTerms: 	GOBPTerm, GOMFTerm, GOCCTerm environments built.



We will demonstrate the gene IDs mapping process using the genes PROBED ID of the DE analysis result saved in submodule 2. To do that, we first need to download the analysis result from Google Cloud Bucket. We will use commands similar to those presented in previous submodules to download and load the DE results to generate the list of genes to be mapped.

In [5]:
# Download the limma_results.rds file to the "data" folder in current directory
system("gsutil cp gs://ccpa_bucket/limma_results.rds ./data/")
# Loading the DE result
data = readRDS("./data/limma_results.rds")

By default, the DE analysis performed by `limma` contains multiple features. However, adjusted *p-value* and *gene ID* are the most important features for enrichment analysis. We can use the following code to list all of the gene IDs and their *p-value*.

In [6]:
# Get p-value from DE results
genelist <- data$pFDR
# Assign gene IDs to associated p-values
names(genelist) <- data$PROBEID

After successfully obtaining the gene list, we need to map the gene IDs to the gene symbols using `hgu133plus2.db`.

In [7]:
# Map gene IDs to gene symbols using the "select" method of AnnotationDbi
gene <- suppressMessages(AnnotationDbi::select(hgu133plus2.db, keys = names(genelist), columns = "SYMBOL"))

In [8]:
head(gene)

,PROBEID,SYMBOL
,<chr>,<chr>
1,222178_s_at,NA
2,224687_at,ANKIB1
3,207488_at,NA
4,239226_at,NA
5,234109_x_at,ONECUT3
6,212833_at,SLC25A46


In [9]:
# Remove duplicated gene IDs
gene <- gene[!duplicated(gene[, 1]), ]

# Assign the result to a new gene list with gene symbol
geneList2 <- genelist
names(geneList2) <- gene[, 2]

Now, we can search for related GO terms based on the new gene list using `topGO` package. First, we need to create a `topGOdata`
object.

In [10]:
# Retrieve all the GO terms related to the gene list obtained from the expression matrix
GOdata <- new("topGOdata", description = "", ontology = "BP", 
              allGenes = geneList2, geneSel = function(x) x, nodeSize = 10, 
              annot = annFUN.org, ID = "alias", mapping = "org.Hs.eg")


Building most specific GOs .....

	( 12272 GO terms found. )


Build GO DAG topology ..........

	( 15583 GO terms and 35092 relations. )


Annotating nodes ...............

	( 16314 genes annotated to the GO terms. )



We can search for related GO terms using `geneInTerm` function and view the term with associated genes.

In [11]:
# Obtain a list of genes for each GO term
allGO = genesInTerm(GOdata)
# show the first 5 GO terms
allGO[1:5]

$`GO:0000002`
 [1] "AKT3"     "DNA2"     "DNAJA3"   "ENDOG"    "FLCN"     "LIG3"    
 [7] "LONP1"    "MDP1"     "MEF2A"    "METTL4"   "MGME1"    "MPV17"   
[13] "OPA1"     "PARP1"    "PIF1"     "PIM1"     "POLB"     "POLG"    
[19] "POLG2"    "PPARGC1A" "PRIMPOL"  "RRM1"     "RRM2B"    "SESN2"   
[25] "SLC25A33" "SLC25A36" "SLC25A4"  "SSBP1"    "STOX1"    "TOP3A"   
[31] "TP53"     "TWNK"     "TYMP"    

$`GO:0000003`
   [1] "A1CF"         "A2M"          "AAAS"         "ABAT"        
   [5] "ABCC8"        "ABHD2"        "ACE"          "ACE2"        
   [9] "ACOD1"        "ACOX1"        "ACR"          "ACRBP"       
  [13] "ACRV1"        "ACSBG2"       "ACSL4"        "ACTL7A"      
  [17] "ACTL9"        "ACTR2"        "ACTR3"        "ACVR1"       
  [21] "ACVR1B"       "ACVR1C"       "ACVR2A"       "ADA"         
  [25] "ADAD1"        "ADAD2"        "ADAM15"       "ADAM18"      
  [29] "ADAM2"        "ADAM20"       "ADAM21"       "ADAM28"      
  [33] "ADAM29"       "ADAM30"       "ADAM32"       "ADAMTS1"     
  [37] "ADAMTS16"     "ADAMTS2"      "ADCY10"       "ADCY3"       
  [41] "ADCY7"        "ADCYAP1"      "ADCYAP1R1"    "ADGB"        
  [45] "ADGRG1"       "ADGRG2"       "ADIG"         "ADM"         
  [49] "ADNP"         "ADRA2A"       "ADRA2B"       "AFF4"        
  [53] "AFG2A"        "AFP"          "AGFG1"        "AGO2"        
  [57] "AGO4"         "AGRP"         "AKAP3"        "AKAP4"       
  [61] "AKR1C3"       "AKT1"         "ALDOA"        "ALKBH5"      
  [65] "ALOX15B"      "ALPL"         "AMBP"         "AMD1"        
  [69] "AMH"          "AMHR2"        "ANAPC1"       "ANAPC10"     
  [73] "ANAPC11"      "ANAPC13"      "ANAPC15"      "ANAPC16"     
  [77] "ANAPC2"       "ANAPC4"       "ANAPC5"       "ANAPC7"      
  [81] "ANG"          "ANGPT2"       "ANKLE1"       "ANKRD31"     
  [85] "ANKRD49"      "ANTXR1"       "ANXA1"        "AP3B1"       
  [89] "APC2"         "APELA"        "APLF"         "APOB"        
  [93] "APOL2"        "APOL3"        "APP"          "AR"          
  [97] "AREG"         "ARHGDIB"      "ARID4A"       "ARID4B"      
 [101] "ARID5B"       "ARMC12"       "ARMC2"        "ASB1"        
 [105] "ASF1B"        "ASH1L"        "ASIP"         "ASPM"        
 [109] "ASZ1"         "ATAT1"        "ATM"          "ATN1"        
 [113] "ATP1A1"       "ATP1A4"       "ATP2B2"       "ATP2B4"      
 [117] "ATP7A"        "ATP8B3"       "ATR"          "ATRX"        
 [121] "AURKA"        "AURKC"        "AVP"          "AVPR1A"      
 [125] "AXDND1"       "AXIN1"        "AXL"          "AZI2"        
 [129] "AZIN2"        "B4GALNT1"     "B4GALT1"      "BACH1"       
 [133] "BAG6"         "BAK1"         "BAP1"         "BASP1"       
 [137] "BAX"          "BBS1"         "BBS2"         "BBS4"        
 [141] "BCAP31"       "BCL2"         "BCL2L1"       "BCL2L10"     
 [145] "BCL2L11"      "BCL2L2"       "BCL6"         "BIK"         
 [149] "BIRC3"        "BMAL1"        "BMP15"        "BMP4"        
 [153] "BMP5"         "BMP6"         "BMP7"         "BMPR1A"      
 [157] "BMPR1B"       "BMPR2"        "BNC1"         "BOK"         
 [161] "BOLL"         "BPY2"         "BRCA2"        "BRD2"        
 [165] "BRDT"         "BRINP1"       "BRIP1"        "BRME1"       
 [169] "BSG"          "BTBD18"       "BUB3"         "C14orf39"    
 [173] "C16orf92"     "C1QBP"        "C2CD6"        "C3"          
 [177] "C3orf62"      "C9orf78"      "CA12"         "CABS1"       
 [181] "CABYR"        "CACNA1H"      "CAD"          "CADM1"       
 [185] "CALCA"        "CALR"         "CALR3"        "CAPN2"       
 [189] "CASP2"        "CASP3"        "CASP8"        "CATSPER1"    
 [193] "CATSPER2"     "CATSPER3"     "CATSPERB"     "CATSPERD"    
 [197] "CATSPERE"     "CATSPERG"     "CBL"          "CBS"         
 [201] "CBX2"         "CCDC136"      "CCDC182"      "CCDC34"      
 [205] "CCDC38"       "CCDC39"       "CCDC40"       "CCDC42"      
 [209] "CCDC62"       "CCDC63"       "CCDC87"       "CCIN"        
 [213] "CCNA1"        "CCNB

Now, we already had GO terms with genes. However, we still do not know the meaning of GO terms related to biological processes. We can use `GO.db` database to get a set of annotation maps describing the entire Gene Ontology assembled using data from GO. We can use the following code to install the `GO.db` R package.

In [12]:
suppressMessages({
    if (!require("BiocManager", quietly = TRUE)) {
        install.packages("BiocManager")
        suppressWarnings(BiocManager::install("GO.db", update = F))
    }
})

In [13]:
library(GO.db)

Then, we can use the following commands to obtain the GO terms description.

In [14]:
# Getting the name of each GO term
terms <- names(allGO)
# Getting the description of each GO term
descriptions <- lapply(Term(terms), function(go_term) go_term[[1]])

In [15]:
descriptions[1:10]

$`GO:0000002`
[1] "mitochondrial genome maintenance"

$`GO:0000003`
[1] "reproduction"

$`GO:0000012`
[1] "single strand break repair"

$`GO:0000018`
[1] "regulation of DNA recombination"

$`GO:0000022`
[1] "mitotic spindle elongation"

$`GO:0000027`
[1] "ribosomal large subunit assembly"

$`GO:0000028`
[1] "ribosomal small subunit assembly"

$`GO:0000038`
[1] "very long-chain fatty acid metabolic process"

$`GO:0000041`
[1] "transition metal ion transport"

$`GO:0000045`
[1] "autophagosome assembly"

In order to perform enrichment analysis in later submodules, we need to save the GO terms and genesets to the standard output. One commonly used format is Gene Matrix Transposed file format *(\*.gmt)*. The GMT file format is a tab delimited file format that describes gene sets. In the GMT format, each row represents a gene set; in the GMX format, each column represents a gene set. Here, we can save GO terms and genesets to the *\*.gmt* using the following `writeGMT` function, it takes in a gene set list, a description list of the gene set and a name of the output file.

In [16]:
#' Function to save Gene Ontology (GO) terms with gene sets to local storage in GMT format.
#'
#' @param genesets A list where each element corresponds to a gene set, named by the GO term. Each element contains a character vector of gene symbols belonging to that gene set.
#' @param descriptions A list where each element corresponds to a gene set (GO term) description. It is indexed by the GO term names and contains character descriptions.
#' @param outfile The path to the output file where the GMT-formatted data will be saved.
#' @return This function is used for saving GO terms with gene sets to a GMT-formatted file.

writeGMT <- function(genesets, descriptions, outfile) {
  # Check if the output file already exists, and remove it if it does
  if (file.exists(outfile)) {
    file.remove(outfile)
  }

  # Loop through each gene set (GO term) in the input genesets
  for (gs in names(genesets)) {
    # Prepare a line to write to the output file
    line <- c(gs, gsub("\t", " ", descriptions[[gs]]), genesets[[gs]])

    # Write the line to the output file in GMT format
    # The line contains GO term, description, and the associated gene symbols
    write(line, file = outfile, sep = "\t", append = TRUE, ncolumns = length(genesets[[gs]]) + 2)
  }
}


# Specify the output file path
outfile <- "./data/GO_terms.gmt"
# Call the writeGMT function to save GO terms with genesets
writeGMT(genesets = allGO, descriptions = descriptions, outfile = outfile)

In [17]:
# save the GO terms with genesets to Google Cloud Bucket
# replace <BUCKET_NAME> with name of your bucket that was previously made in submodule 1
# system("gsutil cp ./data/GO_terms.gmt gs://<BUCKET_NAME>", intern = TRUE)
system("gsutil cp ./data/GO_terms.gmt gs://ccpa_bucket", intern = TRUE)

character(0)

## Kyoto Encyclopedia of Genes and Genomes (KEGG)
### Overview
KEGG is a collection of databases dealing with genomes, biological pathways, diseases, drugs, and chemical substances. KEGG is utilized for bioinformatics research and education, including data analysis in genomics, metagenomics, metabolomics and other omics studies, modeling and simulation in systems biology, and translational research in drug development. The KEGG database project was initiated in 1995 by Minoru Kanehisa, professor at the Institute for Chemical Research, Kyoto University, under the then ongoing Japanese Human Genome Program. Foreseeing the need for a computerized resource that can be used for biological interpretation of genome sequence data, he started developing the KEGG PATHWAY database. It is a collection of manually drawn KEGG pathway maps representing experimental knowledge on metabolism and various other functions of the cell and the organism. Each pathway map contains a network of molecular interactions and reactions and is designed to link genes in the genome to gene products (mostly proteins) in the pathway. This has enabled the analysis called KEGG pathway mapping, whereby the gene content in the genome is compared with the KEGG PATHWAY database to examine which pathways and associated functions are likely to be encoded in the genome. KEGG is a "computer representation" of the biological system. It integrates building blocks and wiring diagrams of the system—more specifically, genetic building blocks of genes and proteins, chemical building blocks of small molecules and reactions, and wiring diagrams of molecular interaction and reaction networks. The illustrative structure of KEGG is presented as figure below.
![](./images/Module3/KEGG.jpg)


In [18]:
#Run the following command to take the quiz
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule3-2.html" width=100% height=250></iframe>')

### Retrieving pathways from KEGG databases
In this section, we will retrieve pathways and related genesets from the KEGG database using R commands. Here we will use the `KEGGREST` R package that provides a client interface to the KEGG REST server. `KEGGREST` can be installed from the Bioconductor using the following command.

In [19]:
suppressMessages({
    if (!require("BiocManager", quietly = TRUE)) {
        install.packages("BiocManager")
    }
    suppressWarnings(BiocManager::install("KEGGREST", update = F))
})

In [20]:
suppressPackageStartupMessages({
    library(KEGGREST)
})

KEGG contains a number of databases. To get an idea of what is available, run `listDatabases()`:

In [21]:
KEGGREST::listDatabases()

[1] "pathway"  "brite"    "module"   "ko"       "genome"   "vg"      
 [7] "ag"       "compound" "glycan"   "reaction" "rclass"   "enzyme"  
[13] "disease"  "drug"     "dgroup"   "environ"  "genes"    "ligand"  
[19] "kegg"

We can use these databases in further queries. Note that in many cases you can also use a three-letter KEGG organism code or a “T number” (genome identifier) in the same place you would use one of these database names.

We can obtain the list of organisms available in KEGG with the `keggList()` function:

In [22]:
organism <- keggList("organism")

In [23]:
print(paste0("KEGG supports ", dim(organism)[1], " organisms"))

[1] "KEGG supports 9623 organisms"


To view the supported organisms we can use the following command:

In [24]:
# View several supported organism
head(organism)

T.number,organism,species,phylogeny
T01001,hsa,Homo sapiens (human),Eukaryotes;Animals;Vertebrates;Mammals
T01005,ptr,Pan troglodytes (chimpanzee),Eukaryotes;Animals;Vertebrates;Mammals
T02283,pps,Pan paniscus (bonobo),Eukaryotes;Animals;Vertebrates;Mammals
T02442,ggo,Gorilla gorilla gorilla (western lowland gorilla),Eukaryotes;Animals;Vertebrates;Mammals
T01416,pon,Pongo abelii (Sumatran orangutan),Eukaryotes;Animals;Vertebrates;Mammals
T03265,nle,Nomascus leucogenys (northern white-cheeked gibbon),Eukaryotes;Animals;Vertebrates;Mammals


In [25]:
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule3-3.html" width=100% height=250></iframe>')

In submodule 02, we performed DE analysis on a human dataset. Therefore, we need to download pathways for humans. The abbreviation of human pathway in KEGG is `hsa` and we can use the `keggList` function to get the pathway list.

In [26]:
# Obtain the pathways belong to human
pathways.list <- keggList("pathway", "hsa")

The pathway list contains pathway description and pathway code in a single line of text. To see the first five pathways related to **carbohydrate metabolism** we will use the commands below.

In [27]:
#list the specific pathways to view
pathway_ids <- c("hsa00010", "hsa00020", "hsa00030", "hsa00040", "hsa00051")
#view their names
pathways.list[pathway_ids]

hsa00010 
            "Glycolysis / Gluconeogenesis - Homo sapiens (human)" 
                                                         hsa00020 
               "Citrate cycle (TCA cycle) - Homo sapiens (human)" 
                                                         hsa00030 
               "Pentose phosphate pathway - Homo sapiens (human)" 
                                                         hsa00040 
"Pentose and glucuronate interconversions - Homo sapiens (human)" 
                                                         hsa00051 
         "Fructose and mannose metabolism - Homo sapiens (human)"

To learn more about these pathways you can visit https://www.genome.jp/kegg/pathway.html. 

In [28]:
#Run the following command to take the quiz
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule3-4.html" width=100% height=250></iframe>')

We can see that, in each line, the text in the quotation mark contains pathway information while the later part contains pathway code leading by a prefix `path:`. To get pathway codes from the pathway list, we can use the following commands:

In [29]:
# Retrieve all the pathway IDs belong to human
pathway.codes <- sub("path:", "", names(pathways.list))
pathway.codes

[1] "hsa01100" "hsa01200" "hsa01210" "hsa01212" "hsa01230" "hsa01232"
  [7] "hsa01250" "hsa01240" "hsa00010" "hsa00020" "hsa00030" "hsa00040"
 [13] "hsa00051" "hsa00052" "hsa00053" "hsa00500" "hsa00520" "hsa00620"
 [19] "hsa00630" "hsa00640" "hsa00650" "hsa00562" "hsa00190" "hsa00910"
 [25] "hsa00920" "hsa00061" "hsa00062" "hsa00071" "hsa00100" "hsa00120"
 [31] "hsa00140" "hsa00561" "hsa00564" "hsa00565" "hsa00600" "hsa00590"
 [37] "hsa00591" "hsa00592" "hsa01040" "hsa00230" "hsa00240" "hsa00250"
 [43] "hsa00260" "hsa00270" "hsa00280" "hsa00290" "hsa00310" "hsa00220"
 [49] "hsa00330" "hsa00340" "hsa00350" "hsa00360" "hsa00380" "hsa00400"
 [55] "hsa00410" "hsa00430" "hsa00440" "hsa00450" "hsa00470" "hsa00480"
 [61] "hsa00510" "hsa00513" "hsa00512" "hsa00515" "hsa00514" "hsa00532"
 [67] "hsa00534" "hsa00533" "hsa00531" "hsa00563" "hsa00601" "hsa00603"
 [73] "hsa00604" "hsa00511" "hsa00730" "hsa00740" "hsa00750" "hsa00760"
 [79] "hsa00770" "hsa00780" "hsa00785" "hsa00790" "hsa00670" "hsa00830"
 [85] "hsa00860" "hsa00130" "hsa00900" "hsa00232" "hsa00524" "hsa00980"
 [91] "hsa00982" "hsa00983" "hsa03020" "hsa03022" "hsa03040" "hsa03010"
 [97] "hsa00970" "hsa03013" "hsa03015" "hsa03008" "hsa03060" "hsa04141"
[103] "hsa04130" "hsa04120" "hsa04122" "hsa03050" "hsa03018" "hsa03030"
[109] "hsa03410" "hsa03420" "hsa03430" "hsa03440" "hsa03450" "hsa03460"
[115] "hsa03082" "hsa03083" "hsa03250" "hsa03260" "hsa03271" "hsa03264"
[121] "hsa03265" "hsa03266" "hsa03267" "hsa02010" "hsa04010" "hsa04012"
[127] "hsa04014" "hsa04015" "hsa04310" "hsa04330" "hsa04340" "hsa04350"
[133] "hsa04390" "hsa04392" "hsa04370" "hsa04371" "hsa04630" "hsa04064"
[139] "hsa04668" "hsa04066" "hsa04068" "hsa04020" "hsa04070" "hsa04072"
[145] "hsa04071" "hsa04024" "hsa04022" "hsa04151" "hsa04152" "hsa04150"
[151] "hsa04080" "hsa04060" "hsa04061" "hsa04512" "hsa04514" "hsa04144"
[157] "hsa04145" "hsa04142" "hsa04146" "hsa04140" "hsa04136" "hsa04137"
[163] "hsa04148" "hsa04110" "hsa04114" "hsa04210" "hsa04215" "hsa04216"
[169] "hsa04217" "hsa04115" "hsa04218" "hsa04510" "hsa04520" "hsa04530"
[175] "hsa04540" "hsa04550" "hsa04814" "hsa04810" "hsa04640" "hsa04610"
[181] "hsa04611" "hsa04613" "hsa04620" "hsa04621" "hsa04622" "hsa04623"
[187] "hsa04625" "hsa04650" "hsa04612" "hsa04660" "hsa04658" "hsa04659"
[193] "hsa04657" "hsa04662" "hsa04664" "hsa04666" "hsa04670" "hsa04672"
[199] "hsa04062" "hsa04911" "hsa04910" "hsa04922" "hsa04923" "hsa04920"
[205] "hsa03320" "hsa04929" "hsa04912" "hsa04913" "hsa04915" "hsa04914"
[211] "hsa04917" "hsa04921" "hsa04926" "hsa04935" "hsa04918" "hsa04919"
[217] "hsa04928" "hsa04916" "hsa04924" "hsa04614" "hsa04925" "hsa04927"
[223] "hsa04260" "hsa04261" "hsa04270" "hsa04970" "hsa04971" "hsa04972"
[229] "hsa04976" "hsa04973" "hsa04974" "hsa04975" "hsa04979" "hsa04977"
[235] "hsa04978" "hsa04962" "hsa04960" "hsa04961" "hsa04964" "hsa04966"
[241] "hsa04724" "hsa04727" "hsa04725" "hsa04728" "hsa04726" "hsa04720"
[247] "hsa04730" "hsa04723" "hsa04721" "hsa04722" "hsa04744" "hsa04740"
[253] "hsa04742" "hsa04750" "hsa04360" "hsa04380" "hsa04211" "hsa04213"
[259] "hsa04710" "hsa04713" "hsa04714" "hsa05200" "hsa05202" "hsa05206"
[265] "hsa05205" "hsa05204" "hsa05207" "hsa05208" "hsa05203" "hsa05230"
[271] "hsa05231" "hsa05235" "hsa05210" "hsa05212" "hsa05225" "hsa05226"
[277] "hsa05214" "hsa05216" "hsa05221" "hsa05220" "hsa05217" "hsa05218"
[283] "hsa05211" "hsa05219" "hsa05215" "hsa05213" "hsa05224" "hsa05222"
[289] "hsa05223" "hsa05166" "hsa05170" "hsa05161" "hsa05160" "hsa05171"
[295] "hsa05164" "hsa05162" "hsa05168" "hsa05163" "hsa05167" "hsa05169"
[301] "hsa05165" "hsa05110" "hsa05120" "hsa05130" "hsa05132" "hsa05131"
[307] "hsa05135" "hsa05133" "hsa05134" "hsa05150" "hsa05152" "hsa05100"
[313] "hsa05146" "hsa05144" "hsa05145" "hsa05140" "hsa05142" "hsa05143"
[319] "hsa05310" "hsa05322" "hsa05323" "hsa05320" "hsa05321" "hsa05330"
[325] "hsa05332" "hsa05340" "hsa05010" "hsa05012" "hsa05014" "hsa05016"
[331] "hsa05017" "hsa05020" "hsa05022" "hs

We can use the following command to check how many pathways are available for human.

In [30]:
print(paste0("Number of available pathways for human are: ", length(pathway.codes)))

[1] "Number of available pathways for human are: 357"


We will use the code below to obtain a list of genes and pathway descriptions for all available human pathways. We'll utilize the `keggGet` function for this task. Since this function can provide 10 results at a time, we will pass 10 pathway IDs as input at each step. 

Something to note is that the KEGG database may also provides pathways that hold many sub pathways so when we look up the pathways through keggGet, they may not have any genes listed in that report. This can skew our results so the for loop below takes out any pathway that does not have a Gene section from the list.

In [31]:
# Create indexes for the outer loop
idx4loop <- seq(from = 1, to = length(pathway.codes), by = 9)
idx4loop <- c(idx4loop, length(pathway.codes))
# Create list to store the gene sets
genes.by.pathway <- list()
description.by.pathway <- list()

# Loop through the each index in idx4loop
for (i in 1:length(idx4loop)) {
    if (i < length(idx4loop)) {
        # Get the ten pathways names to query using keggGet
        pathways_names <- pathway.codes[idx4loop[i]: idx4loop[i+1]]
        pw <- keggGet(pathways_names)
        for (j in 1:length(pw)) {
            pw2 <- pw[[j]]
            # If the returned result of a pathway does not have the key "GENE", skip to the next pathway id
            if (is.null(pw2$GENE)) next
            description.by.pathway[[pathways_names[[j]]]] <-  pw2$NAME          

            pw2 <- pw2$GENE[c(FALSE, TRUE)]
            pw2 <- unlist(lapply(strsplit(pw2, split = ";", fixed = T), function(x) x[1]))
            genes.by.pathway[[pathways_names[[j]]]] <- pw2
        } 
    } 
}
                                 

We can view the first five pathways with their genesets using the following command

In [32]:
# View the five pathways with the genesets
genes.by.pathway[1:5]

$hsa00010
 [1] "HK3"     "HK1"     "HK2"     "HKDC1"   "GCK"     "GPI"     "PFKM"   
 [8] "PFKP"    "PFKL"    "FBP1"    "FBP2"    "ALDOC"   "ALDOA"   "ALDOB"  
[15] "TPI1"    "GAPDH"   "GAPDHS"  "PGK2"    "PGK1"    "PGAM1"   "PGAM2"  
[22] "PGAM4"   "ENO3"    "ENO2"    "ENO1"    "ENO4"    "PKM"     "PKLR"   
[29] "PDHA2"   "PDHA1"   "PDHB"    "DLAT"    "DLD"     "LDHAL6A" "LDHAL6B"
[36] "LDHA"    "LDHB"    "LDHC"    "ADH1A"   "ADH1B"   "ADH1C"   "ADH7"   
[43] "ADH4"    "ADH5"    "ADH6"    "AKR1A1"  "ALDH2"   "ALDH3A2" "ALDH1B1"
[50] "ALDH7A1" "ALDH9A1" "ALDH3B1" "ALDH3B2" "ALDH3A1" "ACSS1"   "ACSS2"  
[57] "GALM"    "PGM1"    "PGM2"    "G6PC1"   "G6PC2"   "G6PC3"   "ADPGK"  
[64] "BPGM"    "MINPP1"  "PCK1"    "PCK2"   

$hsa00020
 [1] "CS"     "ACLY"   "ACO2"   "ACO1"   "IDH1"   "IDH2"   "IDH3B"  "IDH3G" 
 [9] "IDH3A"  "OGDHL"  "OGDH"   "DLST"   "DLD"    "SUCLG1" "SUCLG2" "SUCLA2"
[17] "SDHA"   "SDHB"   "SDHC"   "SDHD"   "FH"     "MDH1"   "MDH2"   "PC"    
[25] "PCK1"   "PCK2"   "PDHA2"  "PDHA1"  "PDHB"   "DLAT"  

$hsa00030
 [1] "GPI"     "G6PD"    "PGLS"    "H6PD"    "PGD"     "RPE"     "RPEL1"  
 [8] "TKT"     "TKTL2"   "TKTL1"   "TALDO1"  "RPIA"    "SHPK"    "DERA"   
[15] "RBKS"    "PGM1"    "PGM2"    "PRPS1L1" "PRPS2"   "PRPS1"   "RGN"    
[22] "IDNK"    "GLYCTK"  "ALDOC"   "ALDOA"   "ALDOB"   "FBP1"    "FBP2"   
[29] "PFKM"    "PFKP"    "PFKL"   

$hsa00040
 [1] "GUSB"    "KL"      "UGT2A1"  "UGT2A3"  "UGT2B17" "UGT2B11" "UGT2B28"
 [8] "UGT1A6"  "UGT1A4"  "UGT1A1"  "UGT1A3"  "UGT2B10" "UGT1A9"  "UGT2B7" 
[15] "UGT1A10" "UGT1A8"  "UGT1A5"  "UGT2B15" "UGT1A7"  "UGT2B4"  "UGT2A2" 
[22] "UGDH"    "UGP2"    "AKR1A1"  "CRYL1"   "RPE"     "RPEL1"   "XYLB"   
[29] "AKR1B1"  "AKR1B10" "AKR1B15" "DCXR"    "SORD"    "DHDH"    "FGGY"   
[36] "CRPPA"  

$hsa00051
 [1] "MPI"     "PMM2"    "PMM1"    "GMPPB"   "GMPPA"   "GMDS"    "GFUS"   
 [8] "FPGT"    "FCSK"    "ENOSF1"  "HK3"     "HK1"     "HK2"     "HKDC1"  
[15] "PFKM"    "PFKP"    "PFKL"    "FBP1"    "FBP2"    "PFKFB1"  "PFKFB2" 
[22] "PFKFB3"  "PFKFB4"  "TIGAR"   "KHK"     "SORD"    "AKR1B1"  "AKR1B10"
[29] "AKR1B15" "ALDOC"   "ALDOA"   "ALDOB"   "TPI1"    "TKFC"

Use the following command to see the description of the first five pathways

In [33]:
# View the description of the first five pathways
description.by.pathway[names(genes.by.pathway[1:5])]

$hsa00010
[1] "Glycolysis / Gluconeogenesis - Homo sapiens (human)"

$hsa00020
[1] "Citrate cycle (TCA cycle) - Homo sapiens (human)"

$hsa00030
[1] "Pentose phosphate pathway - Homo sapiens (human)"

$hsa00040
[1] "Pentose and glucuronate interconversions - Homo sapiens (human)"

$hsa00051
[1] "Fructose and mannose metabolism - Homo sapiens (human)"

Taking a look at the file since the first 8 pathways do not have any genes listed we can remove those hsa ids.

In [34]:
# Saving the pathway data to the local repository
outfile <- "./data/KEGG_pathways.gmt"
writeGMT(genesets = genes.by.pathway, descriptions = description.by.pathway, outfile = outfile)

In [35]:
# Saving the pathway information to the Google Cloud Bucket
# replace <BUCKET_NAME> with name of your bucket that was previously made in submodule 1
# system("gsutil cp ./data/KEGG_pathways.gmt gs://<BUCKET_NAME>", intern = TRUE)
system("gsutil cp ./data/KEGG_pathways.gmt gs://ccpa_bucket", intern = TRUE)

character(0)

# REACTOME Pathway Database
### Overview

REACTOME is an open-source, open-access pathway database that provides a valuable bioinformatics resource for the visualization, interpretation and analysis of biological pathways. Established in 2003, it is a collaborative project led by Lincoln Stein of OICR, Peter D’Eustachio of NYU Langone Health, Henning Hermjakob of EMBL-EBI, and Guanming Wu of OHSU. REACTOME focuses on reactions as its core unit, where entities like nucleic acids, proteins and small molecules contribute to form dynamic networks of interactions and are grouped into pathways. Covering species-specific pathways, each step undergoes rigorous experimental verification, ensuring reliability. This curation process mirrors the editing of a scientific review which requires the participation of multiple domain experts. REACTOME annotates human biological processes by breaking them down into molecular events, resembling classical chemical reactions. Each REACTOME event involves input entities interacting to produce output entities, encompassing reactions like metabolic conversions, binding events, complex formation, transport events, and protein activation. Events are organized into pathways and physical entities, whether small (e.g., glucose) or large (e.g., DNA), are cross-referenced to external databases. Since subcellular localization is crucial in human biological process regulations, Molecules within the Reactome database are linked to particular locations. Therefore, REACTOME treats its instances in different locations as distinct entities. This database uses Gene Ontology to control the vocabularies used to describe the subcellular locations of molecules and reactions, molecular functions, and the larger biological processes that a specific reaction is part of.

REACTOME is broadly used by clinicians, geneticists, genomics researchers, and molecular biologists to interpret the findings of high-throughput experimental studies. It is also employed by computational biologists seeking to develop novel algorithms for leveraging knowledge from genetic research. Additionally, systems biologists use REACTOME to construct predictive models of normal and pathological pathways.

### Retrieving pathways from REACTOME databases

In this section, we'll employ the ReactomeContentService4R package to fetch pathways and their corresponding genes. The R package ReactomeContentService4R offers a convenient way to interact with the Reactome Content Service API. Its predefined functions enable users to access information and images containing proteins, pathways, and other molecules associated with a particular gene or entity in Reactome.

We first need to install and import the `ReactomeContentService4R` package. If you encounter any error while installing this package in the notebook, you can still install it via the terminal. Normally, this package installation requires the `magick` package to be installed. We can install the `magick` package via the terminal using this command: ```sudo apt-get install libmagick++-dev```. To create a terminal in a Vertex AI notebook, you can click on File -> New -> Terminal.

**Note**: Please be aware that the installation instructions provided here might change in the future as the software or package evolves. It is advisable to check for the most up-to-date installation instructions on [CRAN official document page](https://cran.r-project.org/web/packages/magick/vignettes/intro.html). This ensures that you have the latest and most accurate information for installing the package. 


In [38]:
# Install and load required packages with suppressWarnings and suppressMessages

# install.packages("magick")
suppressWarnings({
    suppressMessages({
        if (!require("BiocManager", quietly = TRUE))
            install.packages("BiocManager")

        BiocManager::install("ReactomeContentService4R", force = TRUE)
    })
})



In [39]:
library("ReactomeContentService4R")


Connecting...
welcome to Reactome v87!



To retrieve the infomation of pathways of specific species, we could use the `getSchemaClass` function. The `getSchemaClass` function fetch Instances by Class, to use this function, we need to pass the following arguments:

- `class`: A character string specifying the schema class name.
- `species`: A character string specifying the name or taxon id or dbId or abbreviation of species. Only Event and PhysicalEntity classes can specify species
- `all`: A boolean to decide to return ALL entries or not, default is FALSE. 

This function returns a sorted dataframe containing entries that belong to the specified schema class.

In the following code, we are getting all human pathways information.

In [40]:
pathways <- getSchemaClass(class = "Pathway", species = "human", all = TRUE)
pathways[1:5]

Total 2673 entries of Pathway with species Homo sapiens, retrieving 2673 of them...




  |                                                                      |   0%

dbId,displayName,stId,stIdVersion,isInDisease,isInferred,name,releaseDate,speciesName,hasDiagram,hasEHLD,className,schemaClass,doi,releaseStatus,definition
<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<list>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>
15869,Metabolism of nucleotides,R-HSA-15869,R-HSA-15869.7,FALSE,FALSE,Metaboli....,2004-07-06,Homo sapiens,TRUE,TRUE,Pathway,Pathway,NA,NA,NA
68616,Assembly of the ORC complex at the origin of replication,R-HSA-68616,R-HSA-68616.6,FALSE,FALSE,Assembly....,2004-07-06,Homo sapiens,FALSE,FALSE,Pathway,Pathway,10.3180/R-HSA-68616.3,NA,NA
68689,CDC6 association with the ORC:origin complex,R-HSA-68689,R-HSA-68689.6,FALSE,FALSE,CDC6 ass....,2004-07-06,Homo sapiens,FALSE,FALSE,Pathway,Pathway,NA,NA,NA
68867,Assembly of the pre-replicative complex,R-HSA-68867,R-HSA-68867.10,FALSE,FALSE,Assembly....,2004-07-06,Homo sapiens,FALSE,FALSE,Pathway,Pathway,10.3180/R-HSA-68867.7,NA,NA
68875,Mitotic Prophase,R-HSA-68875,R-HSA-68875.7,FALSE,FALSE,Mitotic ....,2004-07-06,Homo sapiens,TRUE,FALSE,Pathway,Pathway,NA,NA,The initial stage of mitosis in which the chromosomes are condensed but are not yet attached to a mitotic spindle.


We will select only the pathway ID and their associated name column from the information dataframe.

In [41]:
reactome_pathways <- pathways[, c("stId", "displayName")]
head(reactome_pathways)

stId,displayName
<chr>,<chr>
R-HSA-15869,Metabolism of nucleotides
R-HSA-68616,Assembly of the ORC complex at the origin of replication
R-HSA-68689,CDC6 association with the ORC:origin complex
R-HSA-68867,Assembly of the pre-replicative complex
R-HSA-68875,Mitotic Prophase
R-HSA-68877,Mitotic Prometaphase


Next, We will get all genes ID for each pathway and store them in a list just like what we did with KEGG and GO Terms. To do that, we can use the code below.

In [42]:
# Create the geneset list 
rgenes.by.pathway <- sapply(reactome_pathways$stId, function (pathwayID) {
    genesID <- event2Ids(event.id = pathwayID)
    geneSymbol <- genesID[["geneSymbol"]]
    geneSymbol
})


rdescription.by.pathway <- as.list(reactome_pathways$displayName)
names(rdescription.by.pathway) <- reactome_pathways$stId

In [43]:
# Print out some first pathways and their genes IDs
rgenes.by.pathway[1:5]

$`R-HSA-15869`
  [1] "RRM1"             "RR1"              "RRM2"            
  [4] "RR2"              "GLRX"             "GRX"             
  [7] "DCTD"             "RRM2B"            "P53R2"           
 [10] "TXN"              "TRDX"             "TRX"             
 [13] "TRX1"             "GUK1"             "GMK"             
 [16] "GMPK"             "NME1"             "NDPKA"           
 [19] "NM23"             "NME2"             "NM23B"           
 [22] "CMPK1"            "CMK"              "CMPK"            
 [25] "UCK"              "UMK"              "UMPK"            
 [28] "AK1"              "NME4"             "NM23D"           
 [31] "AK8"              "C9orf98"          "AK7"             
 [34] "AK9"              "AKD1"             "AKD2"            
 [37] "C6orf199"         "C6orf224"         "AK5"             
 [40] "NME2P1"           "NME3"             "DTYMK"           
 [43] "CDC8"             "TMPK"             "TYMK"            
 [46] "DCTPP1"           "XTP3TPA"          "CDA03"           
 [49] "CTPS2"            "TXNRD1"           "GRIM12"          
 [52] "KDRF"             "CTPS1"            "CTPS"            
 [55] "AK6"              "CINAP"            "AD-004"          
 [58] "CGI-137"          "AK4"              "AK3"             
 [61] "AK3L1"            "NUDT13"           "TYMS"            
 [64] "TS"               "OK/SW-cl.29"      "DUT"             
 [67] "GSR"              "GLUR"             "GRD1"            
 [70] "AK2"              "ADK2"             "XDH"             
 [73] "XDHA"             "ITPA"             "C20orf37"        
 [76] "My049"            "OK/SW-cl.9"       "PNP"             
 [79] "NP"               "NT5C1B"           "AIRP"            
 [82] "FKSG85"           "NT5C2"            "NT5B"            
 [85] "NT5CP"            "PNT5"             "NT5C1A"          
 [88] "GDA"              "KIAA1258"         "NUDT15"          
 [91] "MTH2"             "NUDT18"           "MTH3"            
 [94] "NUDT1"            "MTH1"             "NUDT1"           
 [97] "MTH1"             "NUDT1"            "MTH1"            
[100] "NUDT1"            "MTH1"             "NUDT9"           
[103] "NUDT10"           "PSEC0099"         "UNQ3012/PRO9771" 
[106] "NUDT16"           "ADPRM"            "C17orf48"        
[109] "MDS006"           "Nbla03831"        "NUDT5"           
[112] "NUDIX5"           "HSPC115"          "DNPH1"           
[115] "C6orf108"         "RCL"              "NT5E"            
[118] "NT5"              "NTE"              "NT5C"            
[121] "DNT1"             "UMPH2"            "POMP"            
[124] "NT5C3A"           "NT5C3"            "P5N1"            
[127] "UMPH1"            "HSPC233"          "TYMP"            
[130] "ECGF1"            "UPP1"             "UP"              
[133] "UPP2"             "NT5M"             "DNT2"            
[136] "DPYD"             "DPYS"             "UPB1"            
[139] "BUP1"             "AGXT2"            "AGT2"            
[142] "ENTPD1"           "CD39"             "ENTPD3"          
[145] "CD39L3"           "ENTPD7"           "LALP1"           
[148] "ENTPD6"           "CD39L2"           "IL6ST2"          
[151] "ENTPD4"           "KIAA0392"         "LALP70"          
[154] "LYSAL1"           "ENTPD8"           "UNQ2492/PRO5779" 
[157] "ENTPD2"           "CD39L1"           "ENTPD5"          
[160] "CD39L4"           "PCPH"             "SAMHD1"          
[163] "MOP5"             "DHODH"            "CAD"             
[166] "UMPS"             "OK/SW-cl.21"      "GMPS"            
[169] "IMPDH2"           "IMPD2"            "IMPDH1"          
[172] "IMPD1"            "ATIC"             "PURH"            
[175] "OK/SW-cl.86"      "ADSS"             "ADSS2"           
[178] "ADSSL1"           "ADSS1"            "ADSL"            
[181] "AMPS"             "PAICS"            "ADE2"            
[184] "AIRC"             "PAIS"             "GART"            
[187] "PGFT"             "PRGS"             "PPAT"            
[190] "GPAT"    

In [44]:
# Print out some first pathways and their descriptions
rdescription.by.pathway[1:10]

$`R-HSA-15869`
[1] "Metabolism of nucleotides"

$`R-HSA-68616`
[1] "Assembly of the ORC complex at the origin of replication"

$`R-HSA-68689`
[1] "CDC6 association with the ORC:origin complex"

$`R-HSA-68867`
[1] "Assembly of the pre-replicative complex"

$`R-HSA-68875`
[1] "Mitotic Prophase"

$`R-HSA-68877`
[1] "Mitotic Prometaphase"

$`R-HSA-68881`
[1] "Mitotic Metaphase/Anaphase Transition"

$`R-HSA-68882`
[1] "Mitotic Anaphase"

$`R-HSA-68884`
[1] "Mitotic Telophase/Cytokinesis"

$`R-HSA-68886`
[1] "M Phase"

Now, we can save the REACTOME pathways to a gmt file for later uses with the `writeGMT` function.

In [45]:
# Saving the pathway information to the local repository
outfile <- "./data/REACTOME_pathways.gmt"
writeGMT(genesets = rgenes.by.pathway, descriptions = rdescription.by.pathway, outfile = outfile)


In [46]:
# Saving the pathway information to the Google Cloud Bucket
# replace <BUCKET_NAME> with name of your bucket that was previously made in submodule 1
# system("gsutil cp ./data/REACTOME_pathways.gmt gs://<BUCKET_NAME>", intern = TRUE)
system("gsutil cp ./data/REACTOME_pathways.gmt gs://ccpa_bucket", intern = TRUE)

character(0)

In the next submodule, we will do Pathway Analysis.


In [47]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 11 (bullseye)

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.13.so;  LAPACK version 3.9.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] ReactomeContentService4R_1.10.0 KEGGREST_1.42.0                
 [3] hgu133plus2.db_3.13.0           org.Hs.eg.db_3.18.0            
 [5] topGO_2.54.0                    SparseM_1.81                